In [41]:
import requests 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time 
import re 
from tqdm.notebook import tqdm
import random

import shutil #이미지저장

import pandas as pd
import numpy as np
import os

import re #숫자만 남기기

In [2]:
#pip install selenium

In [3]:
#pip install webdriver-manager

오류 수정 내용 : 
1. 페이지 요청 오류 났을 시(너무 많은 요청 or 삭제된 계정) 모든 반환값을 -1로 저장
2. timestamp 안뜨는거 수정
3. 모든 반복문에 tqdm바(진행 상태 확인) 추가

In [42]:
def get_Posts_Followers_Followings(): #게시글 수, 팔로워 수, 팔로윙 수
    
    source = driver.page_source
    soup = BeautifulSoup(source, 'html.parser')
    #title = soup.find('title').text
    profile_countable_list = []
    
    try:
        #태그 위치
        li_tags = soup.select('.container ul > li')

        for li in li_tags:
            text = li.text.replace(' ','') #공백 지우기
            counts = int(re.sub('[^0-9]','',text)) #숫자만 냄구기
            profile_countable_list.append(counts)
        if len(profile_countable_list)<3:
            profile_countable_list = ['-','-','-']
    except:
        print('get_Posts_Followers_Followings 예외 발생!')
        profile_countable_list = ['-','-','-']  
    
    return profile_countable_list

In [3]:
def get_InfluId_Biography_ProfileUrl(): #id, biography, 프로필사진url
    
    
    #title = soup.find('title').text
    source = driver.page_source
    soup = BeautifulSoup(source, 'html.parser')   
    profile_uncountable_list = []
    
    try: 
        #container = soup.select_one('.container a')['href']
        container = soup.select_one('.container div.user__title > a').attrs['href']
        profile_uncountable_list.append(container[3:])


        biography = soup.select_one('.container .user__info-desc').text
        profile_uncountable_list.append(biography)


        user_img = soup.select_one('.container .user .user__img')
        user_img_s = str(user_img)
        left = user_img_s.find('url(')
        right = user_img_s.rfind(')')
        profile_uncountable_list.append(user_img_s[left+5:right-1])
        if len(profile_uncountable_list)<3:
            profile_uncountable_list = ['-','-','-']

    except:
        print('get_InfluId_Biography_ProfileUrl 예외 발생!')
        profile_uncountable_list = ['-','-','-']
        
    return profile_uncountable_list

In [43]:
def get_Followers_dict(url, followers):
    new_url = url + '/followers'    
    driver = webdriver.Chrome(executable_path= r'C:/Users/학생/Downloads/chromedriver_win32/chromedriver', chrome_options=options)
    #driver = webdriver.Chrome(executable_path= r'C:/Users/ourus/Python_KPC/Insta_Crawling/chromedriver_win32/chromedriver', chrome_options=options)
    driver.get(new_url)
    
    #title = soup.find('title').text
    
    try:  
        time.sleep(0.5)
        count = []
        tmp = []
        f_var = 0
        followers_dict = {} #딕셔너리형태로 저장
        
        with tqdm(total=followers) as pbar:
            while True:
                
                if (len(count)-len(tmp)) > 200: #200번 스크롤 내려도 개수 그대로인 경우
                    break

                source = driver.page_source
                soup = BeautifulSoup(source, 'html.parser')

                a_tag = soup.select('.container .content__list.followers.grid.infinite_scroll a')

                for a in a_tag:
                    id_url = a.attrs['href'] #https://dumpor.com/v/계정
                    followers_dict[id_url[21:]] = a.img['src'] # {계정이름 : 프로필이미지 url}

                driver.execute_script("window.scrollTo(0, document.body.scrollHeight)") #휠스크롤

                time.sleep(0.5)
                count.append('1')
                
                if f_var != len(followers_dict):
                    f_var = len(followers_dict)
                    tmp.append('1')
                    pbar.update(1)
                print('count:',len(count))
                print('tmp:  ',len(tmp))
                print('len(dict):',len(followers_dict))
                print('-------------------------------')
                
    except:
        return {'-':'-'}
    

    driver.close()
    return followers_dict

In [44]:
def time_change(x,nomalize = False):
    import datetime
    from datetime import date, datetime, timedelta
#     from datetime import timedelta,datetime
    from dateutil.relativedelta import relativedelta
    '''검색결과에 있는 문자를 날짜로 변경
    크롤링시 작성일자가 1분 전/1시간 전/1일 전으로 나타는 경우가 존재
    이러한 패턴을 일반적인 YY-MM-DD형태로 변환
    그리고 월별 계산이 중요하므로 모든 데이터는 1일로 통일
    
    Return
    -------
    x : datetime
        날짜형형태로 전환
        
    Example
    -------
    >>> x = "47분 전"
    >>> time_change(x)
    datetime.datetime(2020, 6, 1, 0, 0)
    
    >>> x2 = "2019.01.31"
    >>> time_change(x2)
    datetime.datetime(2019, 1, 1, 0, 0)
    '''
    
    number = re.compile('[0-9]+')
    today = date.today()#datetime.datetime.today().date()
    x = str(x)
    # 일자
    if " day" in x :
        d = re.sub(r'[^0-9]', '', x)#re.findall(number, x)[0]
        x = today - timedelta(days=int(d))
    elif " hour" in x:
        d = re.sub(r'[^0-9]', '', x)#re.findall(number, x)[0]
        x = today
    elif " minute" in x:
        d = re.sub(r'[^0-9]', '', x)#re.findall(number, x)[0]
        x = today
    #elif x == " month":
    #    x = today - timedelta(days=1)
    elif " month" in x: # '%Y년 %m월 %d일'형태
        m = re.sub(r'[^0-9]', '', x)
        x = today - relativedelta(months=int(m))
        #x = str(new_date.date())
    elif " year" in x: # '%Y년 %m월 %d일'형태
        y = re.sub(r'[^0-9]', '', x)
        x = today - relativedelta(years=int(y))
        #x = str(new_date.date())
    else :
        x = x
    
    #모든일자를 1일로 통일
    if nomalize:
        x = datetime.datetime.strptime(x.strftime('%Y-%m')+"-01","%Y-%m-%d")
    return x

In [45]:
def get_date():
    
    title = soup.find('title').text
    try:
        dates_text = soup.select('#card-page > .container .content__btns > div')[2].span.text
        return dates_text
    except:
        return -1

In [46]:
def get_post_url(posts):
    #url 수집
    url_halfs = []
    count = 0
    with tqdm(total=posts) as pbar:
        while True:     
            if count > posts:
                break

            source = driver.page_source
            soup = BeautifulSoup(source, 'html.parser')

            title = soup.find('title').text      
            try:
                #url저장
                url_source = soup.select('.container a')

                for url_half in url_source:
                    if '/c/' in str(url_half):
                        url_halfs.append(url_half.attrs['href'])           

                #중복제거
                url_halfs = list(set(url_halfs))
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight)") #휠스크롤
                time.sleep(0.5)
                count = count + 1
                pbar.update(1)
                #print(count)
            except:
                url_halfs.append('-1')
    print(f'수집된 url수:{len(url_halfs)}')
    return url_halfs

In [47]:
def get_post_text():
    main_text = soup.select_one('#card-page div.main__list').text
    return main_text

In [48]:
def get_likes_replies(): #[좋아요 개수, 댓글 수]
    likes_replies = []
    print("좋아요,댓글,날짜,다운로드",len(soup.select('#card-page > .container .content__btns > div')))
    try:
        likes_replies.append(soup.select('#card-page > .container .content__btns > div')[0].span.text)
        likes_replies.append(soup.select('#card-page > .container .content__btns > div')[1].span.text)
    except:
        print('get_likes_replies 예외발생')
        likes_replies.append(-1)
        likes_replies.append(-1)
    return likes_replies
    

In [49]:
def get_reply_info(replies): #댓글username : 댓글text
    replies_dict = {} #딕셔너리형태로 저장
    count = 0
    with tqdm(total=int(replies)) as pbar:
        while True:
            if (count > int(float(replies)*0.9)) or (len(replies_dict) > int(replies)): #걍 스크롤 50번이면 댓글 다 가져올거야 제발
                break

            source = driver.page_source
            soup = BeautifulSoup(source, 'html.parser')

            title = soup.find('title').text
            try:
                tag = soup.select('#card-page > .container .wrap.comments.mb-4 > div > div > div > div')
                replies_dict = {}
                text_list = []
                for a in tag:
                    n = 1
                    for b in a: #두개의 태그가 있어서..
                        n = n+1
                        text = b.text
                        if n%2==0: #id
                            text_list.append(text[1:])
                        else: #댓글
                            text_list.append(text)
            except:
                replies_dict = {'-':'-'}
                break
                

            
            replies_dict={text_list[i]: text_list[i + 1] for i in range(0, len(text_list), 2)}
            print("수집한 댓글 수 : ",len(replies_dict)) #확인해 몇개 받았는지
            
            pbar.update(1)
            
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight)") #휠스크롤

            time.sleep(1)
            count = count + 1
    
    return replies_dict

In [50]:
from selenium import webdriver

#add
### 15202623986    seulgistyles_
### 2043464728      style_mans
### 317091289        diegoosorio1
### 30316172873    joysstyles_
### 191403723        kostelevision
### 22566228417    k_fashion_looks
### 38980901318    dailyfashion_news
### 35641698457    irenebaefashion
### 1394688376      bsha_bsha
### 1297647769      coolguy9283
### 1996261596      style_88_

In [61]:
keyword = "k_fashion_looks" 
before_data= keyword + '_게시글.csv'
url = "https://dumpor.com/v/{}/".format(keyword)
os.makedirs('./{}'.format(keyword),exist_ok=True) #하위폴더 생성
os.makedirs('./{}/comms'.format(keyword),exist_ok=True) #하위폴더 생성
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}

In [58]:
#driver options
options = Options()
options.add_argument('--start-maximized') #전체화면
options.add_argument(f'user-agent={headers}')
options.add_argument('--headless') #headless(화면안켜고)

In [59]:
driver = webdriver.Chrome(executable_path= r'C:/Users/학생/Downloads/chromedriver_win32/chromedriver', chrome_options=options)
driver.get(url) 


<ipython-input-59-295a3b4b3ff0>:1: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path= r'C:/Users/학생/Downloads/chromedriver_win32/chromedriver', chrome_options=options)


In [60]:
temp = get_Posts_Followers_Followings()
temp1 = get_InfluId_Biography_ProfileUrl()
profile = {}

profile['user_name'] = temp1[0]    #계정
profile['biography'] = temp1[1]    #프로필
profile['media_count'] = temp[0]    #게시글수
profile['followed_by_count'] = temp[1] #팔로워수
profile['follow_count'] = temp[2]    #팔로윙수
profile['profile_pic_url'] = temp1[2] #프로필사진url

df = pd.DataFrame([profile])
df.to_csv('./{}/{}_profile.csv'.format(keyword,keyword), encoding='utf-8-sig')
df

,user_name,biography,media_count,followed_by_count,follow_count,profile_pic_url
0,k_fashion_looks,💃🏻 20-30대 여성들의 패션 미디어 콘텐츠👜 스타가 착용한 가방정보 : @k_f...,432,19965,1,https://cdn1.dumpor.com/view?q=%3DMDOmZmY30DZp...


In [18]:
#시간 각오하고 돌려야 됨
temp2 = get_Followers_dict(url, temp[1])
follower = temp2

df1 = pd.DataFrame(follower.items(), columns=['follower_id', 'profile_img_url'])

  0%|          | 0/30086 [00:00<?, ?it/s]

count: 1
tmp:   1
len(dict): 12
-------------------------------
count: 2
tmp:   1
len(dict): 12
-------------------------------
count: 3
tmp:   1
len(dict): 12
-------------------------------
count: 4
tmp:   2
len(dict): 24
-------------------------------
count: 5
tmp:   2
len(dict): 24
-------------------------------
count: 6
tmp:   2
len(dict): 24
-------------------------------
count: 7
tmp:   2
len(dict): 24
-------------------------------
count: 8
tmp:   2
len(dict): 24
-------------------------------
count: 9
tmp:   3
len(dict): 36
-------------------------------
count: 10
tmp:   3
len(dict): 36
-------------------------------
count: 11
tmp:   4
len(dict): 48
-------------------------------
count: 12
tmp:   4
len(dict): 48
-------------------------------
count: 13
tmp:   4
len(dict): 48
-------------------------------
count: 14
tmp:   4
len(dict): 48
-------------------------------
count: 15
tmp:   5
len(dict): 60
-------------------------------
count: 16
tmp:   5
len(dict): 60
-

count: 125
tmp:   18
len(dict): 213
-------------------------------
count: 126
tmp:   18
len(dict): 213
-------------------------------
count: 127
tmp:   18
len(dict): 213
-------------------------------
count: 128
tmp:   19
len(dict): 225
-------------------------------
count: 129
tmp:   19
len(dict): 225
-------------------------------
count: 130
tmp:   19
len(dict): 225
-------------------------------
count: 131
tmp:   19
len(dict): 225
-------------------------------
count: 132
tmp:   19
len(dict): 225
-------------------------------
count: 133
tmp:   19
len(dict): 225
-------------------------------
count: 134
tmp:   19
len(dict): 225
-------------------------------
count: 135
tmp:   19
len(dict): 225
-------------------------------
count: 136
tmp:   19
len(dict): 225
-------------------------------
count: 137
tmp:   19
len(dict): 225
-------------------------------
count: 138
tmp:   19
len(dict): 225
-------------------------------
count: 139
tmp:   19
len(dict): 225
------------

In [19]:
df1

,follower_id,profile_img_url
0,hyunwriter,https://cdn2.dumpor.com/view?q=%3D%3DgNhBDZ1UW...
1,mon_achat_,https://cdn1.dumpor.com/view?q=00yNiN2YtIjLqx2...
2,guccheeseee,https://cdn1.dumpor.com/view?q=00yNiN2YtIjLqx2...
3,angelineteo19,https://cdn1.dumpor.com/view?q=00yNiN2YtIjLqx2...
4,jordann.loves.cats.and.dogs,https://cdn2.dumpor.com/view?q=%3D%3DgNhBDZ1UW...
...,...,...
338,ilove5363you,https://cdn2.dumpor.com/view?q=%3D%3DANtcjYjNW...
339,ngocanhpham150,https://cdn2.dumpor.com/view?q=%3D%3DgNhBDZ1UW...
340,ss.3.27,https://cdn2.dumpor.com/view?q=%3D%3DgNhBDZ1UW...
341,holaochakk,https://cdn2.dumpor.com/view?q=%3D%3DgNhBDZ1UW...


In [203]:
for row in tqdm(df1.iloc[:,0]):

    print(row)

dizm007
borckrene.parada
say_lovng_tme
yongpari_
jookkani
foot_ballector
05kim_pro14
p__jaewon
my.raculum
gkrmtlrm
jaesung123
ixthxmxd
ejunh23
jaehwan86
1been_37
piniyua12i77
city_in_seok
yeongryun91
kim_do_yoon1
pol_vengers_j
junopoly00
basement_freestyleclub
youra.0131
ikjunzzz
chefjun1121
jetty.0318
won2wonnie
minn._.seok
dkdlej88
valkyrie_x_rose
x_km.han_x
suriaan_
strawberry12244
jejuddakseawoo_paju
starmom_18
adidascokr
it_ji_yeon0130
cho94_28
gssj5030
chic_w.y
woosik__e
oct.enth
re4mo_myteamulsan
bogus.hoon
intaeee
khpark510
kimobcd
boombapgto
augustinus
xinjimaye
footballshirtthreads
go_d_jisoo
pedrosilva_stz
johee_joy
bagoggyeong146
gjfc__
kop._.chi
kimkimkimjiji
simkungdaddy
pog_js
best_fs__yongin
tomo.sports.club
mongjjjjjjj
76a_0b
jessicashih47
foriser_sport
channnn_ho
sseoyuvi
pumafootball_kr
l_want_b
s.minnns
gng_insta
wfc_bucheon
puma_rc17
coreaseller
d.woong
cr990221
dongzhe_in_japan
asiakiller_yzj
mok_ys
rollercost3
jungcm0414
kop_drm
sasa_footballshirts
wassiddassi
ra

In [23]:
f_biography = []
f_media_count = []
f_followed_by_count = []
f_follow_count = []
driver = webdriver.Chrome(executable_path= r'C:/Users/학생/Downloads/chromedriver_win32/chromedriver', chrome_options=options)
#driver = webdriver.Chrome(executable_path= r'C:/Users/ourus/Python_KPC/Insta_Crawling/chromedriver_win32/chromedriver', chrome_options=options)
for row in tqdm(df1.iloc[:,0]):
#for row in tqdm(df1.iloc[185:,0]):
    print(row)
    temp_url = "https://dumpor.com/v/{}/".format(row)
    print(temp_url)
    driver.get(temp_url)
#     source = driver.page_source
#     soup = BeautifulSoup(source, 'html.parser')
    
    temp2_0 = get_Posts_Followers_Followings()
    print(temp2_0)
    print('-------------------------')
     
    temp2_1 = get_InfluId_Biography_ProfileUrl()
    print(temp2_1[1])
    
    f_biography.append(temp2_1[1])
    f_media_count.append(temp2_0[0])
    f_followed_by_count.append(temp2_0[1])
    f_follow_count.append(temp2_0[2])
df1['f_biography'] = f_biography
df1['f_media_count'] = f_media_count
df1['f_followed_by_count'] = f_followed_by_count
df1['f_follow_count'] = f_follow_count
df1.to_csv('./{}/{}_followers.csv'.format(keyword,keyword), encoding='utf-8-sig')
df1

<ipython-input-23-f3bd151c536b>:5: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path= r'C:/Users/학생/Downloads/chromedriver_win32/chromedriver', chrome_options=options)


  0%|          | 0/343 [00:00<?, ?it/s]

hyunwriter
https://dumpor.com/v/hyunwriter/
[100, 103, 418]
-------------------------
사랑, 건강, 가족 그리고 행복 가장 중요한 건강에 관한 이야기
mon_achat_
https://dumpor.com/v/mon_achat_/
[0, 0, 20]
-------------------------

guccheeseee
https://dumpor.com/v/guccheeseee/
[0, 0, 223]
-------------------------

angelineteo19
https://dumpor.com/v/angelineteo19/
[0, 34, 134]
-------------------------

jordann.loves.cats.and.dogs
https://dumpor.com/v/jordann.loves.cats.and.dogs/
[59, 65, 458]
-------------------------
"Life is a soup and i am the fork"'I live for two reason1. I was born2. I have not died yet
lathifah1602
https://dumpor.com/v/lathifah1602/
[33, 417, 1428]
-------------------------

make.a.deal_order
https://dumpor.com/v/make.a.deal_order/
[0, 44, 608]
-------------------------

_daisy_jennie_
https://dumpor.com/v/_daisy_jennie_/
[1, 54, 72]
-------------------------
Jensetter@jennierubyjane @lesyeuxdenini
cilantro813
https://dumpor.com/v/cilantro813/
[6, 1, 34]
-------------------------

0h1yn
ht

[49, 96, 1415]
-------------------------
me ,myself and i🥳 15 aug 1997🤘🏻INNER CIRCLEsn91 | tu82 • l'arts • psychology🎓
_eggy26
https://dumpor.com/v/_eggy26/
[118, 13, 136]
-------------------------
/拜託讓我住自九
xzvuynie
https://dumpor.com/v/xzvuynie/
[0, 0, 4]
-------------------------

krystallaatte
https://dumpor.com/v/krystallaatte/
[3, 145, 2860]
-------------------------
aku orangnya magerann jd gausa di follow ya
athuthebear
https://dumpor.com/v/athuthebear/
[2, 11, 392]
-------------------------
be my bear, be my seulgi 🐻want sth like seulrene.
kmhno
https://dumpor.com/v/kmhno/
[3, 22, 75]
-------------------------

study_sstudy
https://dumpor.com/v/study_sstudy/
[0, 5, 113]
-------------------------

bui__my_linh
https://dumpor.com/v/bui__my_linh/
[1, 51, 3101]
-------------------------
...
d_es__pair
https://dumpor.com/v/d_es__pair/
[0, 0, 340]
-------------------------

worthwhilll
https://dumpor.com/v/worthwhilll/
[0, 0, 37]
-------------------------
💖
dear._.jw0219
https://dump

[1, 8, 111]
-------------------------
很無聊不要看
kkamo49
https://dumpor.com/v/kkamo49/
[1, 35, 165]
-------------------------
Đời vui thì nsao nà 🤷
kevsertek9
https://dumpor.com/v/kevsertek9/
[0, 2, 348]
-------------------------
İÜ HUKUK
derpsit
https://dumpor.com/v/derpsit/
[1, 6, 368]
-------------------------

hasna_maurent17
https://dumpor.com/v/hasna_maurent17/
[50, 180, 1357]
-------------------------
🐕🍓🐰🐹🐶🐴💚
douceline2709
https://dumpor.com/v/douceline2709/
[0, 5, 308]
-------------------------

yookee1717
https://dumpor.com/v/yookee1717/
[567, 135, 819]
-------------------------
#일상을 담다.#아이와 함께 일상을 담다. #순간순간 행복과 기쁨이 넘쳐넘쳐.#올 한 해도 무사히 보내길 기도하며...
vanillemuse
https://dumpor.com/v/vanillemuse/
[0, 10, 32]
-------------------------

gabby__heart
https://dumpor.com/v/gabby__heart/
[3, 4113, 436]
-------------------------
𝙷𝚒𝚒 𝚐𝚞𝚢𝚜!!😜🖤 17 𝚢𝚎𝚊𝚛𝚜🖤 𝙶𝚎𝚖𝚒𝚗𝚒𝚊𝚗𝚊 🖤 𝙹𝙿-𝙿𝙱🖤 𝟹𝟷/𝟶𝟻🖤𝙴𝚜𝚌𝚛𝚎𝚟𝚎𝚛, 𝚌𝚊𝚗𝚝𝚊𝚛 𝚎 𝚕𝚎𝚛 𝚜ã𝚘 𝚊 𝚖𝚒𝚗𝚑𝚊 𝚙𝚊𝚣.✨🦋••••••••••••••••𝚃𝚑𝚎 𝚠𝚘𝚛𝚜𝚝 𝚕𝚘𝚜𝚜 𝚒𝚜, 𝚝𝚑𝚎 𝚕𝚘𝚜𝚜 𝚘𝚏 𝚝𝚒𝚖𝚎.✨🌻
affectioncircle
https://

[1, 181, 206]
-------------------------
野田 03 👱🏻‍♀️
i_gaby7
https://dumpor.com/v/i_gaby7/
[5, 12, 119]
-------------------------
🌇🐻🐰🐢🐥🦄
ejiongxi
https://dumpor.com/v/ejiongxi/
[2, 1, 30]
-------------------------

daymin_e
https://dumpor.com/v/daymin_e/
[114, 1, 34]
-------------------------
×
sheryyun4
https://dumpor.com/v/sheryyun4/
[0, 6, 482]
-------------------------

david_alex_f
https://dumpor.com/v/david_alex_f/
[9, 204, 781]
-------------------------
*ORTHOPEDIC SURGEON DOCTOR**US MILITARY DOCTOR*
m_ean_ha
https://dumpor.com/v/m_ean_ha/
['-', '-', '-']
-------------------------
get_InfluId_Biography_ProfileUrl 예외 발생!
-
huyeghsis
https://dumpor.com/v/huyeghsis/
[21, 16, 227]
-------------------------

mesikyyyeah
https://dumpor.com/v/mesikyyyeah/
[4, 3, 44]
-------------------------
The whole universe is the tenderness I give you*
tmj777_7
https://dumpor.com/v/tmj777_7/
[0, 0, 793]
-------------------------

minimalmoeum
https://dumpor.com/v/minimalmoeum/
[18, 0, 23]
----------

[0, 32, 195]
-------------------------
vivre d'amour et d'eau fraîche •ᴗ•
nchch.07
https://dumpor.com/v/nchch.07/
[0, 0, 102]
-------------------------

sn__.7
https://dumpor.com/v/sn__.7/
[12, 279, 311]
-------------------------
@nct @redvelvet.smtown
vagienne
https://dumpor.com/v/vagienne/
[1750, 33, 364]
-------------------------
just very hateful
hititdrummer
https://dumpor.com/v/hititdrummer/
[2, 0, 60]
-------------------------

heyhey_hyerim
https://dumpor.com/v/heyhey_hyerim/
[25, 71, 129]
-------------------------
비공개지만.. Ⓕ︎🅞︎Ⓛ︎Ⓛ︎🅞︎🅦︎ 걸면.. 다 받아줌.....앗!! 곰돌이푸!!! 신발보다 귀엽다! @myfavorite_pooh
itsjieying_
https://dumpor.com/v/itsjieying_/
[99, 34, 126]
-------------------------
gimme pho
likvil88
https://dumpor.com/v/likvil88/
[0, 27, 401]
-------------------------
bitter-ie.sweet-ie👾
inmyboring
https://dumpor.com/v/inmyboring/
[4, 1, 273]
-------------------------
この世の真実はじひょたんのみ
bsight_official
https://dumpor.com/v/bsight_official/
[226, 132, 444]
-------------------------
Made in 

,follower_id,profile_img_url,f_biography,f_media_count,f_followed_by_count,f_follow_count
0,hyunwriter,https://cdn2.dumpor.com/view?q=%3D%3DgNhBDZ1UW...,"사랑, 건강, 가족 그리고 행복 가장 중요한 건강에 관한 이야기",100,103,418
1,mon_achat_,https://cdn1.dumpor.com/view?q=00yNiN2YtIjLqx2...,,0,0,20
2,guccheeseee,https://cdn1.dumpor.com/view?q=00yNiN2YtIjLqx2...,,0,0,223
3,angelineteo19,https://cdn1.dumpor.com/view?q=00yNiN2YtIjLqx2...,,0,34,134
4,jordann.loves.cats.and.dogs,https://cdn2.dumpor.com/view?q=%3D%3DgNhBDZ1UW...,"""Life is a soup and i am the fork""'I live for ...",59,65,458
...,...,...,...,...,...,...
338,ilove5363you,https://cdn2.dumpor.com/view?q=%3D%3DANtcjYjNW...,,0,25,766
339,ngocanhpham150,https://cdn2.dumpor.com/view?q=%3D%3DgNhBDZ1UW...,,1,39,262
340,ss.3.27,https://cdn2.dumpor.com/view?q=%3D%3DgNhBDZ1UW...,,14,226,223
341,holaochakk,https://cdn2.dumpor.com/view?q=%3D%3DgNhBDZ1UW...,,3,1,79


In [92]:
driver.close()

In [40]:
#driver = webdriver.Chrome(executable_path= r'C:/Users/ourus/Python_KPC/Insta_Crawling/chromedriver_win32/chromedriver', chrome_options=options)
driver = webdriver.Chrome(executable_path= r'C:/Users/학생/Downloads/chromedriver_win32/chromedriver', chrome_options=options)
driver.get(url) 
temp3 = get_post_url(temp[0])

<ipython-input-40-eef86e2c67c2>:2: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path= r'C:/Users/학생/Downloads/chromedriver_win32/chromedriver', chrome_options=options)


  0%|          | 0/486 [00:00<?, ?it/s]

수집된 url수:486


In [41]:
url_ = {'url': temp3 }
df2 = pd.DataFrame(url_)
df2.to_csv('./{}/{}_게시글url.csv'.format(keyword,keyword), encoding='utf-8-sig')
df2

,url
0,/c/7563617639798798632
1,/c/0997070348449261532
2,/c/4230868397882862452
3,/c/0167820861083207422
4,/c/5078140813975139352
...,...
481,/c/7061897370449429452
482,/c/5308378165519496222
483,/c/7907136821227042612
484,/c/3294731229976167712


In [62]:
driver.close()

In [ ]:
#게시글 정보(이것도 각오하고 돌려야됨)
import datetime
from datetime import date, datetime, timedelta

b_url = pd.read_csv('./{}/{}_게시글url.csv'.format(keyword,keyword))['url']
b_url = b_url.apply(lambda x : x.replace('https://dumpor.com',''))

url_s = pd.Series(temp3)
url_halfs_list = list(url_s[(url_s.apply(lambda x : any([url in x for url in b_url])))].values)

#드라이버 재시작
#driver = webdriver.Chrome(executable_path= r'C:/Users/ourus/Python_KPC/Insta_Crawling/chromedriver_win32/chromedriver', chrome_options=options)
driver = webdriver.Chrome(executable_path= r'C:/Users/학생/Downloads/chromedriver_win32/chromedriver', chrome_options=options)

#각 게시글에서 정보추출
nonetype = type(print()) #타입비교 용도
index_dict = {} #데이터 임시저장
url_titles = []

for index, url_half in tqdm(enumerate(url_halfs_list)):
    #저장 초기화
    contents = {}
    #contents['shortcode'] = [] #계정은 위에서 받아놓았음 
    contents['timestamp'] = [] #
    contents['text'] = []
    contents['comment_count'] = []
    contents['liked_by_count'] = []
    contents['url'] = []
    real_url = f'https://dumpor.com{url_half}'
    
    driver.get(real_url)
    source = driver.page_source
    soup = BeautifulSoup(source, 'html.parser')
    
    #사람처럼 행동 random하게 스크롤 작동
    ran_cnt = random.randint(1,2)
    ran_length = random.choice(np.arange(100,200,100))
    for count in range(0,ran_cnt):
        driver.execute_script(f"window.scrollTo(0, {ran_length})")
        ran_length *= 2
#         time.sleep(0.5)
    
    #dates
    try :
        date = get_date()
        timestamp = time_change(date)
    except :
        timestamp = da
        print('timestamp 예외발생')
    
    #본문
    try :
        text = get_post_text()
        #user_id = souop.select('a.sqdOP.yWX7d._8A5w5.ZIAjV')[0].text
    except :
        text = ''
    
    temp4 = get_likes_replies()
    #댓글수
    try :
        comment_count = temp4[1]
    except : 
        comment_count = []
    
    #좋아요수
    try : 
        liked_by_count = temp4[0]
    except:
        liked_by_count = []
        
    #최소화해놓으면 안되고 최소화 하지 않으면 작동안함()
#     random_stop = random.randint(1,2) #인간답게하려고 랜덤으로 2~4초 스탑
#     time.sleep(random_stop)
      
    #저장
    contents['timestamp'] = str(timestamp) #date
    contents['text'] = text
    contents['comment_count'] = comment_count
    contents['liked_by_count'] = liked_by_count
    contents['url'] = real_url
    
    url_title = url_half.replace('/','_')
    
    reply_dict = get_reply_info(temp4[1])
    for i in range(len(reply_dict)):
        url_titles.append(url_title[3:])
    
    df4_1 = pd.DataFrame({'comm_id': url_titles})
    df4_2 = pd.DataFrame(reply_dict.items(), columns = ['comm_by_username', 'comm_text'])
    df4 = pd.concat([df4_1,df4_2], axis = 1)
    df4.to_csv('./{}/comms/{}_댓글.csv'.format(keyword,url_title), encoding='utf-8-sig')
    
    #인덱스맞춰저장
    index_dict[index] = contents

#데이터프레임화
df3 = pd.DataFrame(index_dict)
df3 = df3.T
df3

<ipython-input-63-11dd69dd50a6>:13: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path= r'C:/Users/학생/Downloads/chromedriver_win32/chromedriver', chrome_options=options)


0it [00:00, ?it/s]

좋아요,댓글,날짜,다운로드 4


  0%|          | 0/3 [00:00<?, ?it/s]

수집한 댓글 수 :  1
수집한 댓글 수 :  1
수집한 댓글 수 :  1
좋아요,댓글,날짜,다운로드 4


  0%|          | 0/12 [00:00<?, ?it/s]

수집한 댓글 수 :  6
수집한 댓글 수 :  6
수집한 댓글 수 :  6
수집한 댓글 수 :  6
수집한 댓글 수 :  6
수집한 댓글 수 :  6
수집한 댓글 수 :  6
수집한 댓글 수 :  6
수집한 댓글 수 :  6
수집한 댓글 수 :  6
수집한 댓글 수 :  6
좋아요,댓글,날짜,다운로드 4


  0%|          | 0/469 [00:06<?, ?it/s]

수집한 댓글 수 :  11
수집한 댓글 수 :  11
수집한 댓글 수 :  11
수집한 댓글 수 :  11
수집한 댓글 수 :  11
수집한 댓글 수 :  21
수집한 댓글 수 :  21
수집한 댓글 수 :  21
수집한 댓글 수 :  21
수집한 댓글 수 :  31
수집한 댓글 수 :  31
수집한 댓글 수 :  31
수집한 댓글 수 :  31
수집한 댓글 수 :  42
수집한 댓글 수 :  42
수집한 댓글 수 :  51
수집한 댓글 수 :  51
수집한 댓글 수 :  51
수집한 댓글 수 :  51
수집한 댓글 수 :  61
수집한 댓글 수 :  61
수집한 댓글 수 :  61
수집한 댓글 수 :  61
수집한 댓글 수 :  73
수집한 댓글 수 :  85
수집한 댓글 수 :  85
수집한 댓글 수 :  85
수집한 댓글 수 :  96
수집한 댓글 수 :  96
수집한 댓글 수 :  96
수집한 댓글 수 :  96
수집한 댓글 수 :  96
수집한 댓글 수 :  96
수집한 댓글 수 :  96
수집한 댓글 수 :  106
수집한 댓글 수 :  106
수집한 댓글 수 :  118
수집한 댓글 수 :  128
수집한 댓글 수 :  128
수집한 댓글 수 :  128
수집한 댓글 수 :  128
수집한 댓글 수 :  128
수집한 댓글 수 :  128
수집한 댓글 수 :  128
수집한 댓글 수 :  128
수집한 댓글 수 :  128
수집한 댓글 수 :  128
수집한 댓글 수 :  128
수집한 댓글 수 :  128
수집한 댓글 수 :  128
수집한 댓글 수 :  137
수집한 댓글 수 :  137
수집한 댓글 수 :  147
수집한 댓글 수 :  147
수집한 댓글 수 :  147
수집한 댓글 수 :  147
수집한 댓글 수 :  147
수집한 댓글 수 :  158
수집한 댓글 수 :  158
수집한 댓글 수 :  167
수집한 댓글 수 :  167
수집한 댓글 수 :  177
수집한 댓글 수 :  177
수집한 댓글 수 :  177
수집한 댓글 수 :

In [46]:
df3.to_csv('./{}/{}_게시글정보.csv'.format(keyword,keyword), encoding = 'utf-8-sig')

In [44]:
#데이터프레임화
df3 = pd.DataFrame(index_dict)
df3 = df3.T
df3

,timestamp,text,comment_count,liked_by_count,url
0,,2차 지필평가 끝. 축하해😁 너무 바빴다 오늘...내일은 졸업사진 촬영애들아 잔뜩 ...,9,688,https://dumpor.com/c/7563617639798798632
1,,여자친구가 잘 찍어 줬네요^^.........뻥,28,900,https://dumpor.com/c/0997070348449261532
2,,굄 GOEM가오픈 시작제주시 일도이동 111-1대기고 26기 대기악. 26기 내 발...,35,766,https://dumpor.com/c/4230868397882862452
3,,방학에는 편하게 핑크,7,903,https://dumpor.com/c/0167820861083207422
4,,오늘은 칠판을.. 설치하다. 선생님에 대한 미련을 남기다.굄찮죠?^^3대가 교사인 ...,30,776,https://dumpor.com/c/5078140813975139352
...,...,...,...,...,...
133,,한글날 바라보기.,9,648,https://dumpor.com/c/9369900864201223142
134,,머리 자르면 진짜 잘생겼는데 애들 놀랄까봐 ㅠㅠ 안자르는거 아시죠??ㅎㅎ아 그리고 ...,22,764,https://dumpor.com/c/9059261561130442632
135,,고등학교 교사와 제자가 함께하는 아무노래 챌린지 풀버전😍 제가 몸치 박치여서... ...,230,2107,https://dumpor.com/c/7939120604293518222
136,,12월의 시작 연말 마무리 잘하기,25,1924,https://dumpor.com/c/8090171548703619812


In [53]:
df4

,comment_id,comment_text
0,94zino,날도 좋고 코디도 좋고~~
1,whit.e_100,충분히 행복해 보이는데요😍😍😍
2,_mowky,6월도 행복하기❤️
3,nawhxx,깔끔한 정석룩입니다👏🔥
4,won.e_,행복하세여~~
5,midnight.__.blue,너의 행복 나의 행복 우리의 행복 고기 먹자
6,12_3_o80bin,행복해지기!! 메모!!
7,064_steady,#dailylook #ootd #데일리 #데일리룩 #심플룩 #남친룩 #남자코디 #오...


### 인플루언서
- 계정명     .. O
- 프로필     .. O
- 팔로워수    .. O
- 팔로윙수    .. O
- 비즈니스 계정 ..X
- 카테고리명   ..X
- 프로필사진url .. O
- 게시글 수    .. O

### 팔로워 리스트
- 계정명       ..O
- 프로필사진url ..O

### 포스트(게시글)
- 게시글 id .. X
- 날짜      .. O
- 타입      ..X
- 텍스트    .. O
- 댓글수    .. O
- 좋아요수  .. O
- 이미지캡션   ..X

### 좋아요_클릭
- 게시글id ..X
- 좋아요수  ..O
- 좋아요클릭id .. X

### 댓글
- 게시글id ..X
- 댓글id  .. O
- 댓글내용 .. O

### 팔로워 개인 프로필
- 계정명  .. O
- 팔로워수  .. O
- 팔로윙수  .. O
- 비즈니스 계정 ..X
- 카테고리명  .. X
- 프로필사진url.. O
- 게시글수  .. O

## ----------------------------------------여기까지--------------------------------------------------

In [437]:
url2 = 'https://dumpor.com/v/p_xoun/followers'

In [438]:
url3 = "https://dumpor.com/c/5350319975241489362"
driver = webdriver.Chrome(executable_path= r'C:/Users/ourus/Python_KPC/Insta_Crawling/chromedriver_win32/chromedriver', chrome_options=options)
driver.get(url2)

<ipython-input-438-c7c79097c5e4>:2: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path= r'C:/Users/ourus/Python_KPC/Insta_Crawling/chromedriver_win32/chromedriver', chrome_options=options)


In [321]:
source = driver.page_source
soup = BeautifulSoup(source, 'html.parser')

In [447]:
a_tag = soup.select('.container .content__list.followers.grid.infinite_scroll a')
a_tag[0].img['src']

'https://cdn1.dumpor.com/view?q=%3DYjNiJDOy0DZpN3Xj52XmczQDJkMyEjN9U2bmgjMzUWO3UjN4I2M5MDNkhzY3MTOlhTM3czMhZWY3EGZ9g2bmQTL30jYjNmJBFUQBJ0aUNWLH9UQ90GZlZyXyQULslDWBlEO3N0azkGTYV1M9MGav91Yu9lJ0Vmbu4GZjJmZuEmbm5SMtUzayhmZu0WYydWY0Nnbp1Ddo91Yu91PnBnau42X3ADO2AjM1QTMxgDNyQzN1AjN281M5UjN2UzNzMTNwUDNwQzX3MzNxUzM0EjMvATNxgHM1EzcvkTMtUDO4IjLxUDdvY3L0Vmbu4GZjJmZuEmbm5SMtUzayhmZu0WYydWY0Nnbp9yL6MHc0RHa'

In [448]:
followers_dict={}
count = 0
while True:
    if(count > 1857):
        break

    source = driver.page_source
    soup = BeautifulSoup(source, 'html.parser')

    a_tag = soup.select('.container .content__list.followers.grid.infinite_scroll a')

    for a in a_tag:
        id_url = a.attrs['href'] #https://dumpor.com/v/계정
        followers_dict[id_url[21:]] = a.img['src'] # {계정이름 : 프로필이미지 url}

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)") #휠스크롤

    #followers_dict = set(followers_dict)
    time.sleep(0.5)
    count = count + 1


KeyboardInterrupt: 

In [372]:
tag = soup.select('#card-page > .container .wrap.comments.mb-4 > div > div > div > div')
follwers_dict = {}
id_list = []
text_list = []
for a in tag:

    user_id = a.p.text
    n = 1
    for b in a:
        n = n+1
        text = b.text
        if n%2==0: #id
            text_list.append(text[1:]) 
        else: #댓글
            text_list.append(text)
            
followers_dict={text_list[i]: text_list[i + 1] for i in range(0, len(text_list), 2)}
print(followers_dict)
        


{'muni_koon': '피드 잘보고가요! 자극받고갑니닷🔥소통해요!😊', 'chell_byeol': 'ㅎㅣ 몸매 머선129❤️', 'w_wpppi': '곰도리 두마리🐻🐻\u200d❄️', 'crescentreasures_sidra32': 'Collab? Dm @crescentreasures', '0529.gaeun.ig': '온니 넘 이쁘세요', 'luxeitem_02': '스타일 멋집니당~제 피드도 놀러오세용~^^', 'a_unxa': '인형이냐거', 'ye_e_jin': '아 예뿌다', 'y_one.me': '아 넘이뻐,,,,,,', '__mar_16': '너 혼혈이라며? 천국과 한국 ><', 'vegemmil': 'ㅂㄹ야', 'yu.__.j_01': '와…미췬…아이돌인줄💖', '20._june': '론아 갈 수록 넘 예쁘다 ❤️', 'kim_sj62': '헙 예쁘다', '8.26.seining': '오엠쥐~ 넌 비올때 우산쓰지마라,,,꽃에는 물이 필요하니깐,,,', 'cactus_ze': '헉 여신님 🥰🥰', '1.03lio': '마스크 써야지', 'auaaa.y': '왤케 예뿌냐 진짜', '30s.en': 'ㅈㄴ이뻐 존나 이뻐 존나이뻐 ㅈㄴㅇㅃㅇㅃㅇㅃ'}


In [221]:
#getCount_Posts_Followers_Followings()
#get_InfluId_Biography_ProfileUrl()
#get_Followers_dict(url, followers)


In [279]:
main_text = soup.select_one('#card-page div.main__list').text

In [280]:
main_text

'🌞ㅡ#teamfollowback #맞팔해요 #fllowme#셀스타그램#likeforlikes#얼스타그램 #선팔하면맞팔#전주카페#selca#selfie #팔로우100#뷰티#대전카페#첫줄#좋아요#셀스타그램#헬스#대전프로필#following#필라테스#ootd#청주#오오티디#맞팔#좋반#flf#f#lfl#첫줄반사#lfl#f4f'

In [282]:
dates_text = soup.select_one('#card-page div.main__user-desc').text
dates = dates_text[5+len('ja.lyong'):]
dates

'3 days ago'

'3 days ago'

datetime.date(2021, 8, 14)

In [ ]:
user_id_url

In [ ]:
user_id = user_id_url[3:]
user_id